# Домашняя работа [#6](https://github.com/SlinkoIgor/data_mining_course2/blob/master/05_sorted_json_requests/05_sorted_json_requests_homework.ipynb)

## _imports_

In [1]:
import re
import requests
import pymorphy2 # pip install pymorphy2b

from collections import defaultdict
from pprint import pprint

## 1

Поскольку последующие части скрипта будут тестироваться неоднократно, то сохраним данные 20 первых страниц [liveinternet.ru/rating](https://www.liveinternet.ru/rating) в локальный `.tsv` файл (с предварительной очисткой).

In [2]:
with open('liveinternet_ru.tsv','w', encoding='utf-8') as li20_file:
    for tsv_page in range(1, 21):
        response = requests.get('https://www.liveinternet.ru/rating///today.tsv?page={}'.format(tsv_page))
        response_lines = str(response.content, response.encoding).split('\n')
        li20_file.write('\n'.join(response_lines[1:-1]))

## 2

Некоторые соображения относительно очистки:

1. Просто сразу удалять все  знаки препинания из строки описания не хотелось бы, потому что тогда были бы утеряны такие валидные составные слова, как, например, `mail.ru` или `научно-популярный`. Поэтому из оригинальной строки описания сначала удаляется только часть символов, потом проводится разделение на слова и потом дополнительная очистка уже на уровне слов (например, удаление `'.'` в конце получившегося слова).
2. Использование библиотеки `pymorphy` для лемматизации звучит привлекательно, но проблема в том, что она делает, например, такие замены:
`вконтакте.ру` --> `вконтакте.р`,
`пикабу` --> `пикаба`.
(по крайней мере в самом простом варианте использования морфологического анализа). Поэтому от её использования в таком виде пришлось отказаться.
3. Однако, её оказалось удобно использовать, например, для удаления предлогов, состоящих из более чем одной буквы (`'на'`, `'по'`), союзов и других подобных частей речи.

In [3]:
def extract_words(descr, morphy):
    '''
    Разбивает строку описания сайта на список слов, с очисткой и приведением к нижнему регистру.
    '''
    # разбиение на слова
    words = descr.split()
    
    # очистка отдельных слов и приведение к нижнему регистру, удаление слов из одной буквы (предположительно, предлогов)
    word_cleanup_re = re.compile(r'[.:]$|^\(|\)$')
    words = [ word_cleanup_re.sub('', w.lower()) for w in words if 1 < len(w) ]
    
    # удаление предлогов, частиц, союзов и т.п. 
    return [ w for w in words if morphy.parse(w)[0].tag.POS not in ('PREP','CONJ','PRCL','INTJ') ]

## 3

Проходим по файлу, из строки выделяем описание и число посетителей, разбиваем описание на слова, счётчик каждого слова инкрементируем на полученное число посетителей.

In [4]:
with open('liveinternet_ru.tsv', encoding='utf-8') as li20_file:
    morphy = pymorphy2.MorphAnalyzer()
    descr_cleanup_re = re.compile(r'[№:,«»"]|&\w+;') 

    words_counter = defaultdict(lambda : 0)
    
    for line in li20_file:
        line = line.rstrip().split('\t')
        description = descr_cleanup_re.sub('', line[2])
        count = int(line[3])
        for word in extract_words(description, morphy):
            words_counter[word] += count

## 4 
Сортировка по убыванию значения счётчика слова, вывод топ-100.

In [5]:
pprint( [ ((n+1,) + item) for n, item in enumerate(sorted(words_counter.items(), key=lambda v: -v[1])) if n < 100 ] )

[(1, 'вконтакте.ру', 81373848),
 (2, 'новости', 32875767),
 (3, 'mail.ru', 28045024),
 (4, 'почта', 17182814),
 (5, 'hearst', 12023750),
 (6, 'shkulev', 12023750),
 (7, 'главные', 11514489),
 (8, 'россии', 9760737),
 (9, 'digital', 8878840),
 (10, 'сайт', 7914844),
 (11, 'онлайн', 6960116),
 (12, 'портал', 6738904),
 (13, 'сеть', 6349474),
 (14, 'ответы', 5396840),
 (15, 'украине', 4939033),
 (16, 'видео', 4854040),
 (17, 'мире', 4745894),
 (18, 'риа', 4666508),
 (19, 'комсомольская', 4554890),
 (20, 'часа', 4539807),
 (21, 'network', 4531494),
 (22, 'погода', 4268401),
 (23, 'газета.ru', 4159345),
 (24, 'сообщество', 3925446),
 (25, 'поиск', 3866730),
 (26, 'lenta.ru', 3771123),
 (27, "women's", 3712455),
 (28, 'отзывы', 3684589),
 (29, 'rt', 3617616),
 (30, 'media', 3516492),
 (31, 'канал', 3511868),
 (32, 'все', 3509456),
 (33, 'рамблер/медиа', 3468489),
 (34, 'cобытия', 3468489),
 (35, 'тв', 3367352),
 (36, 'правда-digital', 3338059),
 (37, 'газета', 3282944),
 (38, 'погодный', 292

## 5
Насколько я понимаю, описание сайта на [liveinternet.ru/rating](https://www.liveinternet.ru/rating) - это по сути заголовок (`<title>`) его главной страницы. Так что выводов о популярности каких-то определённых тем в рунете на основе этой информации сделать сложно.

Что касается заданного вопроса, то, честно говоря, не совсем понятно, что здесь имеется в виду. То, что на [liveinternet.ru/rating](https://www.liveinternet.ru/rating) уже есть классификация сайтов по категориям, и эту информацию можно было бы каким-то образом использовать?
В общем, надеюсь, что мы обсудим этот вопрос с Игорем на занятии.